In [1]:
import arcgis
import time

gis = arcgis.GIS('home')

In [2]:
# get the layer for USA structures
item_id_structures = '0ec8512ad21e4bb987d7e848d14e7e24'
item_structures = gis.content.get(item_id_structures)
lyr_structures = item_structures.layers[0]

# get the layer for 2020 wildfires
item_id_wildfires = '37ab7a4a05ff485aba40a53deaa20ca1'
item_wildfires = gis.content.get(item_id_wildfires)
lyr_wildfires = item_wildfires.layers[1]

In [ ]:
fset_single_wildfire = lyr_wildfires.query("FIRE_NAME = 'AVILA'")
fset_single_wildfire

In [4]:
# get the feature from the FeatureSet
wildfire_feature = fset_single_wildfire.features[0]

In [ ]:
# get the geometry from the single feature
wildfire_geom = wildfire_feature.geometry

# get the wildfire name
wildfire_name = wildfire_feature.get_value("FIRE_NAME")

print(wildfire_name)

In [6]:
# Create a spatial filter to find structures that intersect the wildfire
wildfire_filter = arcgis.geometry.filters.intersects(
    wildfire_geom, sr = wildfire_geom['spatialReference']
)

In [ ]:
# Query the structures layer for structures that intersect the wildfire
structures = lyr_structures.query(
    geometry_filter = wildfire_filter,
    return_count_only=True
)
print(structures)

In [8]:
def query_structures_by_wildfire(wildfire_feature,
                                 structures_layer):
    
    try:
        # Get the wildfire geometry and name
        wildfire_geom = wildfire_feature.geometry
        wildfire_name = wildfire_feature.attributes['FIRE_NAME']

        # Create a spatial filter to find structures that intersect the wildfire
        wildfire_filter = arcgis.geometry.filters.intersects(
            wildfire_geom, sr = wildfire_geom['spatialReference']
        )

        # Query the structures layer for structures that intersect the wildfire
        structures = structures_layer.query(
            geometry_filter = wildfire_filter,
            return_count_only=True
        )

        # Return the wildfire name and the number of structures
        return {
            'Wildfire': wildfire_name,
            'Structures': structures
            }
    
    # If an error occurs, return the wildfire name and None for the structures
    except Exception as e:

        # print the error so we know which wildfire failed
        print(wildfire_name, e)

        return {
            'Wildfire': wildfire_name,
            'Structures': None
            }

In [ ]:
fset_single_wildfire = lyr_wildfires.query("FIRE_NAME = 'OAK'")

query_structures_by_wildfire(
    wildfire_feature = fset_single_wildfire.features[0],
    structures_layer = lyr_structures
)

In [10]:
# query all the wildfires
fset_wildfires = lyr_wildfires.query(
    where = "1=1"
)


In [ ]:

all_results = []

# start a timer for the total time
total_start = time.time()

# iterate through the wildfires
for wildfire in fset_wildfires.features:
    
    # timer for individual features
    loop_start = time.perf_counter()
    
    # run the query for each wildfire
    results = query_structures_by_wildfire(
                            wildfire_feature = wildfire,
                            structures_layer = lyr_structures
                        )
    
    all_results.append(results)
    
    # close out the timer
    loop_end = time.perf_counter()
    
    print(results, loop_end - loop_start)
    
# close out the timer for total time
total_end = time.time()
print(total_end - total_start)

In [12]:
import concurrent

In [ ]:
# Start a timer to time the whole operation
mt_start = time.time()

# Create a list to collect all the results
all_results = []

# Use a ThreadPoolExecutor to query structures for each wildfire
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    
    # Create a list to store the future objects
    exec_futures = []

    # Iterate through each wildfire feature
    for wildfire in fset_wildfires.features:

        # Submit a query task for each wildfire
        exec_result = executor.submit(
            query_structures_by_wildfire, # our function
            wildfire_feature = wildfire, # parameters for our function
            structures_layer = lyr_structures
        )

        # Append the future object to the list
        exec_futures.append(exec_result)

    # Iterate through the future objects as they complete
    for f in concurrent.futures.as_completed(exec_futures):
        all_results.append(f.result())

# End timer and print the total time
mt_end = time.time()
print(mt_end - mt_start)

In [14]:
import pandas
df = pandas.DataFrame(all_results)

In [ ]:
df.head()

Copyright 2025 Esri